# Projet de Catégorisation de texte (Feature engineering) - Test Prediction

In [1]:
#!pip install -U textblob
#!pip install -U spacy
#!python -m spacy download en
#!python -m spacy download en_core_web_sm
#ltk.download()
#!python -m textblob.download_corpora
#!pip install gensim
# !pip install mlflow
# !pip install pyngrok

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

from textblob import TextBlob
import nltk

from nltk.corpus import stopwords
import nltk

from nltk.stem import PorterStemmer

import os
if os.path.isdir('./bbcsport-fulltext'):
    print('repertoire existe')

mypath = 'test'

from textblob import Word

import spacy
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from gensim.models import Word2Vec
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline

import mlflow as mlf
import pickle

In [3]:
nltk.download("punkt")
nltk.download('stopwords')
nltk.download('wordnet')
mots_stop = stopwords.words('english')
ps = PorterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mornd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mornd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mornd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
list_files = []
list_contents = []
list_cat = []

for path, subdirs, files in os.walk(mypath):
    for name in files:
        my_path = os.path.join(path, name)
        list_files.append(name.split('.')[0])
        
        with open(my_path, 'r') as file:
            content = file.read()
            list_contents.append(content)
    


In [5]:
df = pd.DataFrame({'filename':list_files, 'content': list_contents})

In [6]:
df

,filename,content
0,003,South Africa far too strong again\n\nSecond on...


In [7]:
df['content'] = df["content"].map(lambda x: TextBlob(x).words)
df["content"] = df["content"].map(lambda line :  [mot.lower() for mot in line if mot.lower() not in mots_stop] )
df["content"] = df["content"].map(lambda line :  [mot for mot in line if re.match(r'^-?\d+(?:\.\d+)$', mot) is None] )
df["content"] = df["content"].map(lambda line :  [mot for mot in line if re.match(r"^\'\w{2}$", mot) is None] )
df["content"] = df["content"].map(lambda line :  [mot for mot in line if re.match(r"^\'\w$", mot) is None] )
df["content"] = df["content"].map(lambda line :  [mot for mot in line if re.match(r"[^@]+@[^@]+\.[^@]+", mot) is None] )
df["content"] = df["content"].map(lambda line :  [mot for mot in line if re.match(r".*\d+.*", mot) is None] )
df["content"] = df["content"].map(lambda line :  [ps.stem(mot) for mot in line] )
df["content"] = df["content"].map(lambda line :  [Word(mot).lemmatize() for mot in line] )
df["content"] = df["content"].map(lambda line : " ".join([mot for mot in line]))

In [8]:
df.drop('filename',axis=1, inplace=True)

## Test de Prediction

In [9]:
#Importation du modèle
if os.path.exists('models/model.pkl'):
    print("Loding Trained Model .... ")
    model = pickle.load(open('models/model.pkl','rb'))

Loding Trained Model .... 


In [10]:
predictions = model.predict([df['content'][0]])

In [11]:
for i in range(0, len(predictions)):
    print("Le fichier fourni appartient à la catégorie : ",predictions[i])

Le fichier fourni appartient à la catégorie :  cricket
